In [ ]:
# https://tarunlalwani.com/post/reusing-existing-browser-session-selenium/
# https://medium.com/better-programming/lets-create-an-instagram-bot-to-show-you-the-power-of-selenium-349d7a6744f7
# https://towardsdatascience.com/web-scraping-using-selenium-and-beautifulsoup-99195cd70a58
# https://medium.com/@srujana.rao2/scraping-instagram-with-python-using-selenium-and-beautiful-soup-8b72c186a058
import time
from bs4 import BeautifulSoup
import bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests

%matplotlib inline
import matplotlib.pyplot as plt
import uuid
import pandas as pd

## Open browser to login page

In [ ]:
email = 'harvestharvest894@gmail.com'
password = 'earplugs99'

browser = webdriver.Chrome()
browser.get('https://www.instagram.com/accounts/login/ ')

## Login

In [ ]:
emailInput = browser.find_elements_by_css_selector('form input')[0]
passwordInput = browser.find_elements_by_css_selector('form input')[1]

emailInput.send_keys(email)
passwordInput.send_keys(password)
passwordInput.send_keys(Keys.ENTER)

time.sleep(2)

## Dismiss notification pop

In [ ]:
browser.find_element_by_xpath('//button[text()="Not Now"]').click()

## Search for profile

In [ ]:
# search_input = browser.find_element_by_xpath("//input[@placeholder='Search']")

In [ ]:
# search_input.send_keys('pizza')
# time.sleep(1)

## Scroll through page and collect source

In [ ]:
SCROLL_PAUSE_TIME = 1.0
soups = []
# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

for _ in range(10):
    soups.append(BeautifulSoup(browser.page_source))
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    
    last_height = new_height

## Grab all posts

In [ ]:
# soup = BeautifulSoup(browser.page_source)

# articles = soup.find_all('article')

In [ ]:
def extract_from_post_article(article: bs4.element.Tag):
    post_images = [image for image in article.find_all('img') if 'profile picture' not in image.get('alt')]
    post_image = post_images[0]
    alt_text = post_image.get('alt')
    img_insta_url = post_image.get('src')
    
    influencer = article.contents[0].text
    
    caption = article.contents[2].find_all(recursive=False)[2].text
    if caption.startswith(influencer):
        caption = caption[len(influencer):]

    caption = caption.split('View all')[0]
    
    id = uuid.uuid4()
    
    r = requests.get(img_insta_url)
    img_file = "{}_{}.jpg".format(influencer, id)
    with open(img_file, 'wb') as f:
        f.write(r.content)
        
    return {
        'alt_text': alt_text,
        'img_insta_url': img_insta_url,
        'influencer': influencer,
        'caption': caption,
        'id': id,
        'img_file': img_file,
    }
    
    
# extract_from_post_article(articles[0])

In [ ]:
def process_soup(soup):
    articles = soup.find_all('article')
    
    processed_articles = []
    for article in articles:
        processed_articles.append(
            extract_from_post_article(article)
        )
    return pd.DataFrame(processed_articles)

article_dfs = [process_soup(soup) for soup in soups]

In [ ]:
pd.concat(article_dfs)